In [ ]:
!pip install python-telegram-bot 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install telegram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install opensmile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

"""
Simple Bot to reply to Telegram messages.
First, a few handler functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.
Usage:
Basic Echobot example, repeats messages.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

import logging
import opensmile
import audiofile
import os
import pandas as pd
import numpy as np
import joblib

#import opensmile
from telegram import Update
from telegram import ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext


# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)
#================================================================================
def ExtraerCaracteristicas (ruta): #ingresa df:Dataframe con los path de las rutas de audio , sale dataAudio con las caracteristicas LowLevelDescriptors
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
    )
  
    dataAudio = pd.DataFrame(columns =['Loudness_sma3','alphaRatio_sma3','hammarbergIndex_sma3','slope0-500_sma3','slope500-1500_sma3', 'spectralFlux_sma3',
                                       'mfcc1_sma3','mfcc2_sma3','mfcc3_sma3','mfcc4_sma3','F0semitoneFrom27.5Hz_sma3nz','jitterLocal_sma3nz',
                                        'shimmerLocaldB_sma3nz', 'HNRdBACF_sma3nz','logRelF0-H1-H2_sma3nz','logRelF0-H1-A3_sma3nz',
                                        'F1frequency_sma3nz', 'F1bandwidth_sma3nz','F1amplitudeLogRelF0_sma3nz','F2frequency_sma3nz',
                                        'F2bandwidth_sma3nz','F2amplitudeLogRelF0_sma3nz','F3frequency_sma3nz','F3bandwidth_sma3nz',
                                        'F3amplitudeLogRelF0_sma3nz'], index = range(1))

    #for n in range(ravdess_df.shape[0]):
    signal, sampling_rate = audiofile.read(ruta ,  always_2d=True)
    fn= smile.process_signal(signal, sampling_rate)
    df_reset=fn.reset_index(drop=True)

    dataAudio.iloc[0] = df_reset.mean()
    return dataAudio
##=======================================================##




#================================================================================

# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True), )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text(update.message.text)

def saludar (update: Update, context: CallbackContext) -> None:
    """Este es un hander  el cual recibe el comando /saludar y muestra un saludo"""
    update.message.reply_text("Hola soy el comando saludar")

def IdentificarImagen (update: Update, context: CallbackContext) -> None:
    """Este es un hander  el cual recibe el comando /saludar y muestra un saludo"""
    photo_file = update.message.photo[-1].get_file()
    photo_file.download('imagen_pruba.jpg')

    ruta = 'input_audio/'+ os.listdir('input_audio')[0]
    dato_audio = ExtraerCaracteristicas(ruta)
    sc = joblib.load('modelos/169_Normalizacion.pkl')
    x_test_std = sc.transform(dato_audio)
    clf = joblib.load('modelos/169_Modelo_Prueba.pkl')
    y_predict = clf.predict(x_test_std)
    y_predict1=np.argmax(y_predict, axis=1)
    if y_predict1[0] == 0:
        update.message.reply_text("Angry")
    elif y_predict1[0] == 1:
        update.message.reply_text('Happy')
    elif y_predict1[0] == 2:
        update.message.reply_text('Neutral')
    else:
        update.message.reply_text('Sad')  
    
    
def audiovoz (update: Update, context: CallbackContext) -> None:
    """Este es un hander  el cual recibe el comando /saludar y muestra un saludo"""
    audio_file = update.message.voice.get_file()
    audio_file.download('input_audio/audio_prueba.mp3')
    update.message.reply_text("Hola soy el Clasificador de emociones")
    ######################################################################
    ruta = 'input_audio/'+ os.listdir('input_audio')[0]
    dato_audio = ExtraerCaracteristicas(ruta)
    sc = joblib.load('modelos/169_Normalizacion.pkl')
    x_test_std = sc.transform(dato_audio)
    clf = joblib.load('modelos/169_Modelo_Prueba.pkl')
    y_predict = clf.predict(x_test_std)
    y_predict1=np.argmax(y_predict, axis=1)
    if y_predict1[0] == 0:
        update.message.reply_text("Angry")
    elif y_predict1[0] == 1:
        update.message.reply_text('Happy')
    elif y_predict1[0] == 2:
        update.message.reply_text('Neutral')
    else:
        update.message.reply_text('Sad')  
    


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("COLOCAR TU TOKEN DE BOT TELEGRAM AQUI")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))
    dispatcher.add_handler(CommandHandler("saludar", callback=saludar))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))
    dispatcher.add_handler(MessageHandler(Filters.photo & ~Filters.command, IdentificarImagen))
    dispatcher.add_handler(MessageHandler(Filters.audio & ~Filters.command, audiovoz))
    dispatcher.add_handler(MessageHandler(Filters.voice & ~Filters.command, audiovoz))
    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()



if __name__ == '__main__':
    main()



2022-07-02 18:48:42,955 - apscheduler.scheduler - INFO - Scheduler started
2022-07-02 18:49:24,971 - sox - WARNING - SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    
2022-07-02 18:50:26,035 - tensorflow - WARNING - 5 out of the last 5 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7fbd8819d9e0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controllin